In [ ]:
# Assuming your kernel autoinstalled renv, uncomment the following code
# to install the rest of the packages
# renv::restore()

## Title
Predicting number of shares an article will get based on the number of links, images and videos in the article.

## Summary
 Our analysis aims to predict the number of shares an article published by Mashable over a period of two years will get based on the number of links, images and videos in the article using knn classification. 

## Introduction
An article is classified as popular if it has equal to or more than 1400 shares and unpopular if it has less than 1400 shares. The model aims to predict whether an article will be popular based on the number of links, images, and videos in it. 
The dataset is publicly available and has 61 attributes out of which we chose 4 including the three predictors and the variable of interest, shares, which refers to the number of shares of the article in social networks.

## Methods and Results
Loading relevant packages.

In [ ]:
library(kknn)
library(renv)
library(tidymodels)
library(tidyverse)
library(repr)

Reading data from dataset.

In [ ]:
url <- "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip"
download.file(url,"articles.zip")
temp_zip <- tempfile()
download.file(url, temp_zip)
# Unzip the file in the data directory
unzip(temp_zip, exdir = "data/")
# Cleanup: Remove the temporary zip file
unlink(temp_zip)
directories <- list.dirs("data/", recursive = FALSE)

# List the files within the first directory, filtering for CSV files
csv_files <- list.files(directories[1], pattern = "\\.csv$", full.names = TRUE)

articles <- read.csv(csv_files, header = TRUE)

Creating a boxplot of the variable of interest

In [ ]:
ggplot(data = articles) +
    geom_boxplot(aes(y = shares)) +
    labs(title = "Boxplot of Shares") +
    ylab(label = 'Shares') +
    theme(axis.title = element_text(size = 20), axis.text = element_text(size = 15), title = element_text(size = 25))

Figure 1. Boxplot of the variable of interest, shares


Obtaining summary statistics about the variable of interest.

In [ ]:
summary(articles$shares)
sd(articles$shares)

The variable of interest, shares, has a median of 1400 and a mean of 3395. The distribution spread is quite large, with a standard deviation of 11627. We will be using knn classification to determine whether an article will be popular. We will create a new variable called is_popular with two values 1 (popular) and 0 (unpopular)". An article is classified as popular if it has equal to or more than 1400 shares and unpopular if it has less than 1400 shares.


In [ ]:
articles_clean <- articles |>
      select(shares, num_hrefs, num_imgs, num_videos) |>
      mutate(is_popular = ifelse(shares < 1400, 0, 1)) |>
      mutate(is_popular = as.factor(is_popular))

head(articles_clean)

In [ ]:
write.csv(articles_clean, 'data/clean_Articles.csv')

Since we will be performing classification, we split our data into a training and a testing set with a 60% proportion.

In [ ]:
set.seed(2024)

articles_data_split <- initial_split(articles_clean, prop = 0.6, strata = is_popular)
articles_training <- training(articles_data_split)
articles_testing <- testing(articles_data_split)

We start our preliminary data analysis process by examining the number of observations we have in the training set for each class.


In [ ]:
num_obs_training <- articles_training |>
      group_by(is_popular) |>
      summarize(n = n()) |>
      mutate(percentage = 100*n/nrow(articles_training))

num_obs_training

Table 1. Class distribution of article popularity in training set


We can see that our data is balanced and since we have close percentages of data in both popular and unpopular classes, with popular holding the majority at 53.4%.

Next, we visualize the distributions of our predictor variables: num_hrefs, num_imgs, and num_videos. We use is_popular as the fill argument.


In [ ]:
# Histogram 1: Distribution of the number of links in article
mean_hrefs_plot <- ggplot(articles_training, aes(x = num_hrefs, fill = is_popular)) +
  geom_histogram(color = "black") +
  labs(title = "Distribution of number of links", 
       x = "Number of links", 
       fill = "Popular")+
       scale_fill_manual(labels = c("Unpopular", "Popular"))   


# Histogram 1: Distribution of the number of images in article
mean_imgs_plot <- ggplot(articles_training, aes(x = num_imgs, fill = is_popular)) +
  geom_histogram(color = "black") +
  labs(title = "Distribution of number of images", 
       x = "Number of images", 
       fill = "Popular")+
       scale_fill_manual(labels = c("Unpopular", "Popular"))       


# Histogram 1: Distribution of the number of videos in article
mean_videos_plot <- ggplot(articles_training, aes(x = num_videos, fill = is_popular)) +
  geom_histogram(color = "black") +
  labs(title = "Distribution of number of videos", 
       x = "Number of videos", 
       fill = "Popular")  +
       scale_fill_manual(labels = c("Unpopular", "Popular"))   

In [ ]:
mean_hrefs_plot

Figure 2. Histogram of number of links in popular and non-Popular articles


In [ ]:
mean_imgs_plot

Figure 3. Histogram of number of images in popular and non-Popular articles


In [ ]:
mean_videos_plot

Figure 4. Histogram of number of videos in popular and non-Popular articles


Next, we start building our knn classification model.

We start with making a recipe using the training data.

In [ ]:
articles_recipe <- recipe(is_popular ~ num_hrefs + num_imgs + num_videos, data = articles_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())
articles_recipe

Next, we build a tuning model for picking the best value of k.


In [ ]:
tune_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

Next, we perform cross-validation and create a workflow that calculate the metrics for each of the K values 1, 6, ..., 46, and then return a data frame that shows the accuracy of each K value.


In [ ]:
set.seed(2023) # set the seed

# cross-validation
articles_vfold <- vfold_cv(articles_training, v = 5, strata = is_popular)

# create a set of K values
kvals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

# data analysis workflow                       
    knn_results <- workflow() |>
    add_recipe(articles_recipe) |>
    add_model(tune_spec) |>
    tune_grid(resamples = articles_vfold, grid = kvals) |>
    collect_metrics()

accuracies <- knn_results |>
    filter(.metric == "accuracy")

print(accuracies)

Table 2. Summary of KNN model accuracy across different K values with 5-fold cross-validation



We now work on making an accuracy vs k plot.


In [ ]:
best_k_plot <- accuracies |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Number of neighbors", y = "Accuracy Estimate") +
    ggtitle("Accuracy Estimates vs. Number of Neighbors")
best_k_plot

Figure 4.  Results from 5-fold cross validation to choose K. (Accuracy was used as the classification metric as K was varied.)


The value of K that generates the highest accuracy is about 50, therefore we choose K = 50 and use that to build the model.


In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 50) |>
    set_engine("kknn") |>
    set_mode("classification")
knn_spec

Next, we use a workflow to fit our model using our training set, predict the labels in our testing set, and incorporate the predictions as the new column .pred_class into the testing set.


In [ ]:
knn_fit <- workflow() |>
    add_recipe(articles_recipe) |>
    add_model(knn_spec) |>
    fit(data = articles_training)

articles_predict <- predict(knn_fit, articles_testing) |>
    bind_cols(articles_testing)

We now look at the accuracy of the classifier using the testing set with the metrics function.


In [ ]:
articles_accuracy <- articles_predict |>
    metrics(truth = is_popular, estimate = .pred_class) |>
    filter(.metric == "accuracy")

articles_accuracy

Table 3. Classifier testing set accuracy

As can be seen, the accuracy of the classifier isn't very high at 54.6%. Below is the confusion matrix of the classifier model.


In [ ]:
confusion_matrix <- articles_predict |>
    conf_mat(truth = is_popular, estimate = .pred_class)
confusion_matrix

Table 4. Confusion matrix of Aarticle popularity predictions on test data

Given that we care more about popular articles with higher shares, we take popular articles as the "positive" observations. The precision is therefore 0.6211 which is slightly higher than the accuracy of 0.546. The recall is 0.3844 which is significantly lower than the accuracy. This suggests that the model is somewhat precise in predicting popular articles but is cautious and misses some actual instances of popular articles in doing so.

## Discussion:
Summary: Our analysis aims to predict the number of shares an article published by Mashable over a period of two years will get based on the number of links, images and videos in the article using knn classification. An article is classified as popular if it has equal to or more than 1400 shares and unpopular if it has less than 1400 shares. Our findings suggest that the model is only somwhat reliable with an accuracy of 0.546 with precision higher than accuracy but recall lower than accuracy.

Are the findings as expected:
Yes, this is what we expected. Given that a basic KNN  model was used for prediction, it's reasonable to assume that this simple model might be slightly underfitting. Consequently, we anticipate achieving improved outcomes by exploring more complex models.

Impact of Findings:
This insight has the potential to refine content creation strategies by enriching our comprehension of the elements that contribute to content's engagement and popularity, such as identifying the ideal balance of links, images, and videos within an article. Adopting this strategy could significantly boost the future articles' shareability.

Future Questions from Analysis:
This analysis might lead to several Future questions. Firstly, whether there are more model models capable of achieving higher precision and recall. Secondly, the impact of the quality of links, images, and videos within articles on their popularity presents another area for in-depth investigation. 

## References
Dataset Source: 
Online news popularity. UCI Machine Learning Repository. (2015, January). https://archive.ics.uci.edu/dataset/332/online+news+popularity 
